<a href="https://colab.research.google.com/github/gauss5930/Natural-Language-Processing/blob/main/XLNet/XLNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import json
import os
import tensorflow as tf
import modeling

def _get_initializer(FLAGS):
  # 변수 초기화
  if FLAGS.init == 'uniform':
    initializer = tf.initializers.random_uniform(
        minval = -FLAGS.init_range,
        maxval = FLAGS.init_range,
        seed = None
    )

  elif FLAGS.init == 'normal':
    initializer = tf.initializers.random_normal(
        stddev = FLAGS.init_std,
        seed = None
    )

  else:
    raise ValueError('Initializer {} not supported'.format(FALGS.init))
  return initializer

class XLNetConfig(object):
  ''' XLNetConfig는 model checkpoint에 특정된 하이퍼 파라미터를 포함하고 있음
  이 하이퍼 파라미터들은 pre-training 시와 fine-tuning 시에 모두 같아야 함

  n_layer: 레이어의 수
  d_model: hidden size
  n_head: attention head의 수
  d_head: 각 attention head의 차원 크기
  d_inner: feed-forward layer에서 hidden size
  ff_activation: 'relu' 또는 'gelu'
  untie_r: attention에서 bias들을 untie할 지 말지 결정
  n_token: vocab_size
  '''

  def __init__(self, FLAGS = None, json_path = None):
    '''
    XLNetConfig 구조
    하나의 FLAGS 또는 json_path는 제공되어야 한다.
    '''

    assert FLAGS is not None or json_path is not None

    self.keys = ['n_layer', 'd_model', 'n_head', 'd_head', 'd_inner', 'ff_activation', 
                 'untie_r', 'n_token']

    if FLAGS is not None:
      self.init_from_flags(FLAGS)

    if json_path is not None:
      self.init_from_json(json_path)

  def init_from_flags(self, FLAGS):
    for key in self.keys:
      setattr(self, key, getattr(FLAGS, key))

  def init_from_json(self, FLAGS):
    with tf.gfile.Open(json_path) as f:
      json_data = json.load(f)
      for key in self.keys:
        setattr(self, key, json_data[key])

  def to_json(self, json_path):
    # XLNetConfig를 json 파일로 저장
    json_data = {}
    for key in self.keys:
      json_data[key] = getattr(self, key)

    json_dir = os.path.dirname(json_path)
    if not tf.gfile.Exists(json_dir):
      tf.gfile.MakeDirs(json_dir)
    with tf.gfile.Open(json_path, 'w') as f:
      json.dump(json_data, f, indent = 4, sort_keys = True)

def create_run_config(is_training, is_finetune, FLAGS):
  kwargs = dict(
      is_training=is_training,
      use_tpu=FLAGS.use_tpu,
      use_bfloat16=FLAGS.use_bfloat16,
      dropout=FLAGS.dropout,
      dropatt=FLAGS.dropatt,
      init=FLAGS.init,
      init_range=FLAGS.init_range,
      init_std=FLAGS.init_std,
      clamp_len=FLAGS.clamp_len
  )

  if not is_finetune:
    kwargs.update(dict(
        mem_len=FLAGS.mem_len,
        reuse_len=FLAGS.reuse_len,
        bi_data=FLAGS.bi_data,
        clamp_len=FLAGS.clamp_len,
        same_length=FLAGS.same_length
    ))

  return RunConfig(**kwargs)

class RunConfig(object):
  '''
  RunConfig는 pre-training과 fine-tuning에서 서로 다른 하이퍼 파라미터를 가져야 함.
  이 하이퍼 파라미터들은 실행할 때마다 변경할 수 있다.
  '''

  def __init__(self, is_training, use_tpu, use_bfloat16, dropout, dropatt,
               init = 'normal', init_range = 0.1, init_std = 0.02, mem_len = None,
               reuse_len = None, bi_data = False, clamp_len = -1, same_length = False):
    '''
    is_training: 학습 모드인지 아닌지 확인
    use_tpu: TPU를 사용할 지 말 지 확인
    use_bfloat16: float32 대신에 bfloat16 사용
    dropout: dropout 비율
    dropatt: attention 확률에 dropout 비율
    init: 초기화 scheme. 'normal' 또는 'uniform' 둘 중 하나
    init_range: [-init_range, init_range]에서 균일한 분포를 사용해서 파라미터를 초기화
      init='uniform'일 때 가장 효과적임
    mem_len: 캐시해둘 토큰의 수
    reuse_len: 캐시되고 향후 재사용될 현재 배치의 토큰 수이다.
    bi_data: 양방향성 입력 파이프라인을 사용할 지 말 지 정함. 
      pre-training 중에는 True를 사용, fine-tuning 중에는 False를 사용
    clamp_len: clamp_len보다 큰 모든 상대 거리를 고정한다다. -1은 클램핑이 없음을 의미한다.
    same_length: 각 토큰에 대해 똑같은 attention length를 사용할 지 말 지 결정
    '''

    self.init = init
    self.init_range = init_range
    self.init_std = init_std
    self.is_training = is_training
    self.dropout = dropout
    self.dropatt = dropatt
    self.use_tpu = use_tpu
    self.use_bfloat16 = use_bfloat16
    self.mem_len = mem_len
    self.reuse_len = reuse_len
    self.bi_data = bi_data
    self.clamp_len = clamp_len
    self.same_length = same_length

class XLNetModel(object):
  # pre-training 및 fine-tuning 중에 사용되는 XLNet 모델의 wrapper이다.

  def __init__(self, xlnet_config, run_config, input_ids, seg_ids, input_mask,
               memes = None, perm_mask = None, target_mapping = None, inp_q = None,
               **kwargs):
    
    initializer = _get_initializer(run_config)

    tfm_args = dict(
        n_token=xlnet_config.n_token,
        initializer=initializer,
        attn_type="bi",
        n_layer=xlnet_config.n_layer,
        d_model=xlnet_config.d_model,
        n_head=xlnet_config.n_head,
        d_head=xlnet_config.d_head,
        d_inner=xlnet_config.d_inner,
        ff_activation=xlnet_config.ff_activation,
        untie_r=xlnet_config.untie_r,

        is_training=run_config.is_training,
        use_bfloat16=run_config.use_bfloat16,
        use_tpu=run_config.use_tpu,
        dropout=run_config.dropout,
        dropatt=run_config.dropatt,

        mem_len=run_config.mem_len,
        reuse_len=run_config.reuse_len,
        bi_data=run_config.bi_data,
        clamp_len=run_config.clamp_len,
        same_length=run_config.same_length
    )

    input_args = dict(
        inp_k=input_ids,
        seg_id=seg_ids,
        input_mask=input_mask,
        mems=mems,
        perm_mask=perm_mask,
        target_mapping=target_mapping,
        inp_q=inp_q
    )

    with tf.variable_scope('model', reuse = tf.AUTO_REUSE):
      (self.output, self.new_mems, self.lookup_table) = modeling.transformer_xl(**tfm_args)

    self.input_mask = input_mask
    self.initializer = initializer
    self.clnet_config = clnet_config
    self.run_config = run_config

  def get_pooled_out(self, summary_type, use_summ_proj = True):
    xlnet_config = self.xlnet_config
    run_config = self.run_config

    with tf.variable_scope('model', reuse = tf.AUTO_REUSE):
      summary = modeling.summarize_sequence(
          summary_type=summary_type,
          hidden=self.output,
          d_model=xlnet_config.d_model,
          n_head=xlnet_config.n_head,
          d_head=xlnet_config.d_head,
          dropout=run_config.dropout,
          dropatt=run_config.dropatt,
          is_training=run_config.is_training,
          input_mask=self.input_mask,
          initializer=self.initializer,
          use_proj=use_summ_proj
      )

    return summary

  def get_sequence_output(self):
    # XLNet의 마지막 레이어의 hidden representation
    
    return self.output

  def get_new_memory(self):
    # 이전 메모리와 현재 input representation을 합친 new memory
    # list의 길이는 n_layer와 같음
    return self.new_mems

  def get_embedding_table(self):
    # embedding lookup table
    # input 레이어와 output 레이어 간의 embedding tie
    return self.lookup_table

  def get_initializer(self):
    # tf initilizer
    # XLNet의 top layer에서 변수들을 초기화하기 위해 사용
    return self.initializer